Dependencies/Required installs

In [1]:
# # Dependencies

# ### Mesa
# %pip install mesa`

# ### Solara [For Visualization]:
# https://solara.dev/documentation/getting_started/installing

# 1) `python -m venv solara-env`
# 2) `source ./solara-env/bin/activate`
# 3) `pip install solara`

# ### networkx

# %pip install networkx`

# ### matplotlib
# %pip install matplotlib`

# ### altair

# %pip install altair`

# # ChatGPT Request Dependencies:

# ### OpenAI
# %pip install openai

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import mesa
import requests
import os
import sys
import random
from openai import OpenAI

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline,
    Trainer, TrainingArguments, 
    DataCollatorWithPadding
)

In [ ]:
# API key steup
key = os.environ["GPT_GAME_KEY"]

client = OpenAI(api_key=key)

# response = client.responses.create(
#     model="gpt-5-mini",
#     input="Pretend to be a League of Legends player. Write something you would say in a chat."
# )

#print(response.output_text)

In [3]:
# Load model from hugging face

# pick type
basic_model_name = "visha007/ToxiMuncher-Lite" #baseline model
pro_model_name = "visha007/ToxiMuncher-Pro" #adapted model

# load model
model_name = basic_model_name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=None)

Device set to use cpu


In [ ]:
chat_message = "I don't like player 1"
message_classified=pipe(chat_message)
message_classified=message_classified[0][0]
message_toxic_score= message_classified["score"]
print(message_toxic_score)

REMOVE_THRESHOLD = 3

1.797668695449829


In [ ]:
HIT_GPT = False
REMOVE_THRESHOLD = 3 # How to define threshold
        
class PlayerAgent(mesa.Agent):
    def __init__(self, model):
        # Pass the parameters to the parent class.
        super().__init__(model)
        self.message_removed = f"[player {self.unique_id}'s message has been removed]"
        self.current_message = ""
        self.toxic_score = -1
        isAgentToxic=random.binomialvariate(n=1,p=0.5)
        if(isAgentToxic == 1):
            self.personality = "rude/irritated"
        else:
            self.personality = "kind"

    def chat(self):
        respond=random.binomialvariate(n=1,p=0.5)
        if(respond == 1):
            self.current_message=self.get_message()
            self.toxic_score = self.model.get_toxic_score(self.current_message)
            isToxic=self.model.isMessageToxic(self.toxic_score)
            self.model.push_to_chat(self.format_message())
            if(isToxic):
                print(self.message_removed)
                self.model.push_to_mod_chat(self.message_removed)
            else:
                print(self.format_message())
                self.model.push_to_mod_chat(self.format_message())

        
        # Decide if player responds
        # Decide what message(s?) to respond to
        # update the chat log here
        if(respond == 0):
            self.current_message=""
        
    def get_message(self):
        prompt = self.get_prompt()
        if(HIT_GPT==True):
            response = client.responses.create(
                model="gpt-5-mini",
                input=prompt
            )
            return response.output_text
        else:
            #print(prompt)
            if(self.personality == "rude/irritated"):
                return "fuck you"
                
            elif(self.personality == "kind"):
                return "Let's breathe, play safe, get vision, group for objectives, and call targets"
        

    def get_prompt(self):
        return f"Pretend to be player {self.unique_id} in a League of Legends game. {model.is_winning()}. Write one {self.personality} message you would say in the game chat. This is the current chat history: {model.chat_log}"

    def format_message(self):
        if(self.current_message==""):
            return ""
        
        return (f"\n player {self.unique_id}: {self.current_message}")
        

class GameModel(mesa.Model):
    
    def __init__(self,num_players,end_game_pts,the_ai_model_name):
        super().__init__()
        self.ai_model_name = the_ai_model_name
        self.chat_log = ""
        self.chat_log_moderated = ""
        self.num_players = num_players
        self.ending_game_points = end_game_pts # the number of points a team needs to have to end the game
        self.datacollector = mesa.DataCollector(
            model_reporters={"Game state":self.is_winning},
            agent_reporters={"personality":"personality","current_message": "current_message", "toxic_score":"toxic_score"}
        )
        # self.chat
        # self.personas
        #self.ai_moderator = AiModAgent.create_agents(self,n=1)
        self.pipe = self.get_pipeline()
        agents=PlayerAgent.create_agents(self,n=self.num_players)
        self.is_game_done=False
        self.points = {
            "team":0,
            "opponent":0}
        
    
    def step(self):
        self.progress_game()
        self.agents.shuffle_do("chat")
        self.datacollector.collect(self)
        game_df=model.datacollector.get_model_vars_dataframe()
        agents_df=model.datacollector.get_agent_vars_dataframe()
        #print(game_df)
        #print(agents_df)
        #print(self.chat_log)
        #print(self.chat_log_moderated)
        
        # dump/collect chat log in data collector
    
    def progress_game(self):
        coin_flip=random.binomialvariate(n=1,p=0.5) # Idea: abstract game into a cointoss game
        #update points based on the game outcome
        if(coin_flip==1):
            self.points["team"] = self.points["team"] + 1
        else:
            self.points["opponent"] = self.points["opponent"] + 1
            
        if((self.points["team"] == self.ending_game_points)or (self.points["opponent"] == self.ending_game_points)):
            self.is_game_done = True

    def is_winning(self):
        if(self.is_game_done == True):
            return "The game is done"
            
        if(self.points["team"]>self.points["opponent"]):
            return "Your team is winning"
        elif(self.points["team"]<self.points["opponent"]):
            return "Your team is losing"
        elif(self.points["team"]==self.points["opponent"]):
            return "The game is tied"

    def get_pipeline(self):
        tokenizer = AutoTokenizer.from_pretrained(self.ai_model_name)
        ai_model = AutoModelForSequenceClassification.from_pretrained(self.ai_model_name)
        pipe = TextClassificationPipeline(model=ai_model, tokenizer=tokenizer, top_k=None)
        return pipe

    def get_toxic_score(self,message):
        message_classified=pipe(message)
        message_classified=message_classified[0][0]
        message_toxic_score= message_classified["score"]
        return message_toxic_score
    
    def isMessageToxic(self, message_toxic_score):
        if(message_toxic_score >= REMOVE_THRESHOLD):
            return True #"AI Moderator: Your message has been removed."

        return False
        
    def push_to_chat(self,message):
        if((message == None) or (message=="")):
            return
        self.chat_log = self.chat_log + (f"\n{message}")
    
    def push_to_mod_chat(self,message):
        if((message == None) or (message=="")):
            return
        self.chat_log_moderated = self.chat_log_moderated + (f"\n{message}")


In [ ]:
basic_model_name = "visha007/ToxiMuncher-Lite" #baseline model
pro_model_name = "visha007/ToxiMuncher-Pro" #adapted model

ai_models = {
    "lite" : "visha007/ToxiMuncher-Lite",
    "pro": "visha007/ToxiMuncher-Pro"
}

# pick model type
ai_model_name = ai_models["pro"]

model = GameModel(5,5,ai_model_name)

while(model.is_game_done==False):
    model.step()

Device set to use cpu



 player 5: Amazing shotcalling and teamwork, everyone — really proud of how we played, let’s keep this up and finish strong!

 player 3: player 3: Stop facechecking and overextending like that — we're winning, not your solo highlight reel.

 player 5: player 5: Good point, player3 — let's stay grouped and play safe. I'll ward and call targets; we can finish this clean. Great job so far, team!

 player 3: Stop facechecking and overextending like that — we're winning, not your solo highlight reel. Play with the team or stop feeding, seriously.

 player 4: player 4: Stop facechecking like an idiot—group up or stop feeding, we're winning despite you.

 player 1: player 1: Stop facechecking like an idiot and feeding — group with us or mute and stop ruining our lead.

 player 3: player 3: Stop facechecking like a headless chicken — group up or stop throwing our lead, you're making this way harder than it needs to be.

 player 4: player 4: Stop facechecking like an idiot — group up or go fee

In [7]:
print(model.points)
game_df=model.datacollector.get_model_vars_dataframe()
agents_df=model.datacollector.get_agent_vars_dataframe()
print(game_df)
print(agents_df)
print(model.chat_log)

{'team': 5, 'opponent': 1}
             Game state
0  Your team is winning
1  Your team is winning
2  Your team is winning
3  Your team is winning
4  Your team is winning
5      The game is done
                 personality  \
Step AgentID                   
1    1        rude/irritated   
     2                  kind   
     3        rude/irritated   
     4        rude/irritated   
     5                  kind   
2    1        rude/irritated   
     2                  kind   
     3        rude/irritated   
     4        rude/irritated   
     5                  kind   
3    1        rude/irritated   
     2                  kind   
     3        rude/irritated   
     4        rude/irritated   
     5                  kind   
4    1        rude/irritated   
     2                  kind   
     3        rude/irritated   
     4        rude/irritated   
     5                  kind   
5    1        rude/irritated   
     2                  kind   
     3        rude/irritated   
     

In [8]:
agents_df=agents_df.rename(columns={"AgentID":"PlayerID"})
agents_df.to_csv("game_output/test.csv")
game_df.to_csv("game_output/game_outcome.csv")

In [9]:
print(model.chat_log)



 player 5: Amazing shotcalling and teamwork, everyone — really proud of how we played, let’s keep this up and finish strong!

 player 3: player 3: Stop facechecking and overextending like that — we're winning, not your solo highlight reel.

 player 5: player 5: Good point, player3 — let's stay grouped and play safe. I'll ward and call targets; we can finish this clean. Great job so far, team!

 player 3: Stop facechecking and overextending like that — we're winning, not your solo highlight reel. Play with the team or stop feeding, seriously.

 player 4: player 4: Stop facechecking like an idiot—group up or stop feeding, we're winning despite you.

 player 1: player 1: Stop facechecking like an idiot and feeding — group with us or mute and stop ruining our lead.

 player 3: player 3: Stop facechecking like a headless chicken — group up or stop throwing our lead, you're making this way harder than it needs to be.

 player 4: player 4: Stop facechecking like an idiot — group up or go fe

In [10]:
print(model.chat_log_moderated)



 player 5: Amazing shotcalling and teamwork, everyone — really proud of how we played, let’s keep this up and finish strong!

 player 3: player 3: Stop facechecking and overextending like that — we're winning, not your solo highlight reel.

 player 5: player 5: Good point, player3 — let's stay grouped and play safe. I'll ward and call targets; we can finish this clean. Great job so far, team!

 player 3: Stop facechecking and overextending like that — we're winning, not your solo highlight reel. Play with the team or stop feeding, seriously.

 player 4: player 4: Stop facechecking like an idiot—group up or stop feeding, we're winning despite you.

 player 1: player 1: Stop facechecking like an idiot and feeding — group with us or mute and stop ruining our lead.

 player 3: player 3: Stop facechecking like a headless chicken — group up or stop throwing our lead, you're making this way harder than it needs to be.

 player 4: player 4: Stop facechecking like an idiot — group up or go fe

In [11]:
with open('game_output/chat_log_moderated.txt', 'w') as output:
    output.write(model.chat_log_moderated)

In [12]:
with open('game_output/chat_log.txt', 'w') as output:
    output.write(model.chat_log)